In [77]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/covid-world-vaccination-progress/country_vaccinations_by_manufacturer.csv
/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv


In [78]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import randint
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import ExtraTreesClassifier

In [79]:
train= pd.read_csv('../input/covid-world-vaccination-progress/country_vaccinations.csv')
train

country iso_code        date  total_vaccinations  \
0      Afghanistan      AFG  2021-02-22                 0.0   
1      Afghanistan      AFG  2021-02-23                 NaN   
2      Afghanistan      AFG  2021-02-24                 NaN   
3      Afghanistan      AFG  2021-02-25                 NaN   
4      Afghanistan      AFG  2021-02-26                 NaN   
...            ...      ...         ...                 ...   
14989     Zimbabwe      ZWE  2021-04-28            458013.0   
14990     Zimbabwe      ZWE  2021-04-29            477597.0   
14991     Zimbabwe      ZWE  2021-04-30            500342.0   
14992     Zimbabwe      ZWE  2021-05-01            520299.0   
14993     Zimbabwe      ZWE  2021-05-02            524199.0   

       people_vaccinated  people_fully_vaccinated  daily_vaccinations_raw  \
0                    0.0                      NaN                     NaN   
1                    NaN                      NaN                     NaN   
2                    NaN                      NaN                     NaN   
3                    NaN                      NaN                     NaN   
4                    NaN                      NaN                     NaN   
...                  ...                      ...                     ...   
14989           388021.0                  69992.0                 24074.0   
14990           400771.0                  76826.0                 19584.0   
14991           414735.0                  85607.0                 22745.0   
14992           428135.0                  92164.0                 19957.0   
14993           430068.0                  94131.0                  3900.0   

       daily_vaccinations  total_vaccinations_per_hundred  \
0                     NaN                            0.00   
1                  1367.0                             NaN   
2                  1367.0                             NaN   
3                  1367.0                             NaN   
4                  1367.0                             NaN   
...                   ...                             ...   
14989             17860.0                            3.08   
14990             17971.0                            3.21   
14991             19194.0                            3.37   
14992             21171.0                            3.50   
14993             19169.0                            3.53   

       people_vaccinated_per_hundred  people_fully_vaccinated_per_hundred  \
0                               0.00                                  NaN   
1                                NaN                                  NaN   
2                                NaN                                  NaN   
3                                NaN                                  NaN   
4                                NaN                                  NaN   
...                              ...                                  ...   
14989                           2.61                                 0.47   
14990                           2.70                                 0.52   
14991                           2.79                                 0.58   
14992                           2.88                                 0.62   
14993                           2.89                                 0.63   

       daily_vaccinations_per_million            vaccines  \
0                                 NaN  Oxford/AstraZeneca   
1                                35.0  Oxford/AstraZeneca   
2                                35.0  Oxford/AstraZeneca   
3                                35.0  Oxford/AstraZeneca   
4                                35.0  Oxford/AstraZeneca   
...                               ...                 ...   
14989                          1202.0   Sinopharm/Beijing   
14990                          1209.0   Sinopharm/Beijing   
14991                          1291.0   Sinopharm/Beijing   
14992                          1424.0   Sinopharm/Beijing   


In [80]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14994 entries, 0 to 14993
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   country                              14994 non-null  object 
 1   iso_code                             14994 non-null  object 
 2   date                                 14994 non-null  object 
 3   total_vaccinations                   9011 non-null   float64
 4   people_vaccinated                    8370 non-null   float64
 5   people_fully_vaccinated              6158 non-null   float64
 6   daily_vaccinations_raw               7575 non-null   float64
 7   daily_vaccinations                   14796 non-null  float64
 8   total_vaccinations_per_hundred       9011 non-null   float64
 9   people_vaccinated_per_hundred        8370 non-null   float64
 10  people_fully_vaccinated_per_hundred  6158 non-null   float64
 11  daily_vaccinations_per_milli

In [81]:
train["date"]= pd.to_datetime(train.date)

 # **Examining features** 

In [82]:
train.describe()

total_vaccinations  people_vaccinated  people_fully_vaccinated  \
count        9.011000e+03       8.370000e+03             6.158000e+03   
mean         4.964584e+06       3.164030e+06             1.580482e+06   
std          2.033452e+07       1.180919e+07             6.840161e+06   
min          0.000000e+00       0.000000e+00             1.000000e+00   
25%          6.000000e+04       5.154925e+04             2.350025e+04   
50%          4.209400e+05       3.255035e+05             1.729175e+05   
75%          1.932222e+06       1.338161e+06             6.887102e+05   
max          2.753380e+08       1.470470e+08             1.047747e+08   

       daily_vaccinations_raw  daily_vaccinations  \
count            7.575000e+03        1.479600e+04   
mean             1.347569e+05        7.769163e+04   
std              5.211912e+05        3.519679e+05   
min              0.000000e+00        0.000000e+00   
25%              2.888500e+03        8.967500e+02   
50%              1.558300e+04        6.178500e+03   
75%              6.249850e+04        2.835000e+04   
max              1.160100e+07        7.205286e+06   

       total_vaccinations_per_hundred  people_vaccinated_per_hundred  \
count                     9011.000000                    8370.000000   
mean                        15.456079                      11.228710   
std                         23.238675                      15.269686   
min                          0.000000                       0.000000   
25%                          1.330000                       1.222500   
50%                          6.300000                       4.825000   
75%                         19.570000                      14.667500   
max                        211.080000                     111.320000   

       people_fully_vaccinated_per_hundred  daily_vaccinations_per_million  
count                           6158.00000                    14796.000000  
mean                               5.66235                     2972.163625  
std                               10.06184                     4720.846894  
min                                0.00000                        0.000000  
25%                                0.63000                      347.000000  
50%                                2.35000                     1475.000000  
75%                                6.30000                     3908.250000  
max                               99.76000                   118759.000000

In [83]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14994 entries, 0 to 14993
Data columns (total 15 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   country                              14994 non-null  object        
 1   iso_code                             14994 non-null  object        
 2   date                                 14994 non-null  datetime64[ns]
 3   total_vaccinations                   9011 non-null   float64       
 4   people_vaccinated                    8370 non-null   float64       
 5   people_fully_vaccinated              6158 non-null   float64       
 6   daily_vaccinations_raw               7575 non-null   float64       
 7   daily_vaccinations                   14796 non-null  float64       
 8   total_vaccinations_per_hundred       9011 non-null   float64       
 9   people_vaccinated_per_hundred        8370 non-null   float64       
 10  people_ful

# **Addressing questions related to vaccines used in each country**

which country used which vaccines?
how many vaccines have been developed?
what is the most commonly used vaccine? 

In [84]:
vaccines=train['vaccines'].unique()
vaccines

array(['Oxford/AstraZeneca',
       'Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, Sputnik V',
       'Sputnik V', 'Oxford/AstraZeneca, Pfizer/BioNTech',
       'Oxford/AstraZeneca, Sinopharm/Beijing, Sputnik V',
       'Pfizer/BioNTech',
       'Johnson&Johnson, Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',
       'Sinovac',
       'Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V',
       'Oxford/AstraZeneca, Sinovac',
       'Moderna, Oxford/AstraZeneca, Pfizer/BioNTech',
       'Oxford/AstraZeneca, Sinopharm/Beijing, Sinovac',
       'Sinopharm/Beijing', 'Pfizer/BioNTech, Sinovac',
       'Sinopharm/Beijing, Sinopharm/Wuhan, Sinovac',
       'Sinopharm/Beijing, Sputnik V', 'Moderna, Pfizer/BioNTech',
       'Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac',
       'Oxford/AstraZeneca, Sinopharm/Beijing',
       'Moderna, Oxford/AstraZeneca',
       'Moderna, Oxford/AstraZeneca, Pfizer/BioNTech, Sinopharm/Beijing, Sputnik V',
       'Covaxin, Oxford/AstraZeneca', 'Oxf

Here we are using string manipulation to identify vaccines uniquely: 

In [85]:
vaccineList=[]
for vaccine in vaccines:
    temp= vaccine.split(', ')
    vaccineList=vaccineList+temp  

In [86]:
vaccineList= pd.unique(vaccineList).tolist()
vaccineList

['Oxford/AstraZeneca',
 'Pfizer/BioNTech',
 'Sinovac',
 'Sputnik V',
 'Sinopharm/Beijing',
 'Johnson&Johnson',
 'Moderna',
 'Sinopharm/Wuhan',
 'Covaxin',
 'CanSino',
 'EpiVacCorona']

In [87]:
print('The total number of vaccines used so far is' ,len(vaccineList))

The total number of vaccines used so far is 11


Group by each country to identify the vaccines used in each country:

In [88]:
vaccines = train.groupby("country").max()['vaccines']
vaccines

country
Afghanistan                                   Oxford/AstraZeneca
Albania        Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...
Algeria                                                Sputnik V
Andorra                      Oxford/AstraZeneca, Pfizer/BioNTech
Angola                                        Oxford/AstraZeneca
                                     ...                        
Venezuela                                              Sputnik V
Vietnam                                       Oxford/AstraZeneca
Wales               Moderna, Oxford/AstraZeneca, Pfizer/BioNTech
Zambia                                        Oxford/AstraZeneca
Zimbabwe                                       Sinopharm/Beijing
Name: vaccines, Length: 195, dtype: object

In [89]:
vaccines_country= vaccines.to_frame()
vaccines_country

vaccines
country                                                       
Afghanistan                                 Oxford/AstraZeneca
Albania      Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac, ...
Algeria                                              Sputnik V
Andorra                    Oxford/AstraZeneca, Pfizer/BioNTech
Angola                                      Oxford/AstraZeneca
...                                                        ...
Venezuela                                            Sputnik V
Vietnam                                     Oxford/AstraZeneca
Wales             Moderna, Oxford/AstraZeneca, Pfizer/BioNTech
Zambia                                      Oxford/AstraZeneca
Zimbabwe                                     Sinopharm/Beijing

[195 rows x 1 columns]

The corresponding vaccines for each country:

In [90]:
count_vaccines = vaccines.str.split(', ')
count_vaccines

country
Afghanistan                                 [Oxford/AstraZeneca]
Albania        [Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac,...
Algeria                                              [Sputnik V]
Andorra                    [Oxford/AstraZeneca, Pfizer/BioNTech]
Angola                                      [Oxford/AstraZeneca]
                                     ...                        
Venezuela                                            [Sputnik V]
Vietnam                                     [Oxford/AstraZeneca]
Wales             [Moderna, Oxford/AstraZeneca, Pfizer/BioNTech]
Zambia                                      [Oxford/AstraZeneca]
Zimbabwe                                     [Sinopharm/Beijing]
Name: vaccines, Length: 195, dtype: object

Count the number of vaccines used in each country: 

In [91]:
country_vaccine= count_vaccines.to_frame()
country_vaccine=country_vaccine.reset_index()
country_vaccine

country                                           vaccines
0    Afghanistan                               [Oxford/AstraZeneca]
1        Albania  [Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac,...
2        Algeria                                        [Sputnik V]
3        Andorra              [Oxford/AstraZeneca, Pfizer/BioNTech]
4         Angola                               [Oxford/AstraZeneca]
..           ...                                                ...
190    Venezuela                                        [Sputnik V]
191      Vietnam                               [Oxford/AstraZeneca]
192        Wales     [Moderna, Oxford/AstraZeneca, Pfizer/BioNTech]
193       Zambia                               [Oxford/AstraZeneca]
194     Zimbabwe                                [Sinopharm/Beijing]

[195 rows x 2 columns]

Number of vaccines each country uses: 

In [92]:
# create Length column
country_vaccine['no_of_vaccines'] = country_vaccine.vaccines.map(len)
country_vaccine.sort_values(by=['no_of_vaccines'], inplace=True, ascending=False)
country_vaccine

country                                           vaccines  \
185  United Arab Emirates  [Oxford/AstraZeneca, Pfizer/BioNTech, Sinophar...   
111                Mexico  [CanSino, Oxford/AstraZeneca, Pfizer/BioNTech,...   
76                Hungary  [Moderna, Oxford/AstraZeneca, Pfizer/BioNTech,...   
97                Lebanon  [Oxford/AstraZeneca, Pfizer/BioNTech, Sinophar...   
85                  Italy  [Johnson&Johnson, Moderna, Oxford/AstraZeneca,...   
..                    ...                                                ...   
66              Gibraltar                                  [Pfizer/BioNTech]   
65                  Ghana                               [Oxford/AstraZeneca]   
63                Georgia                               [Oxford/AstraZeneca]   
62                 Gambia                               [Oxford/AstraZeneca]   
194              Zimbabwe                                [Sinopharm/Beijing]   

     no_of_vaccines  
185               5  
111               5  
76                5  
97                4  
85                4  
..              ...  
66                1  
65                1  
63                1  
62                1  
194               1  

[195 rows x 3 columns]

Number of vaccines each country uses: 

In [93]:
x= country_vaccine.groupby('no_of_vaccines').count()['country'].to_frame()
x

country
no_of_vaccines         
1                    90
2                    49
3                    36
4                    17
5                     3

In [94]:
 country_vaccine.head(20)

country  \
185    United Arab Emirates   
111                  Mexico   
76                  Hungary   
97                  Lebanon   
85                    Italy   
42                  Czechia   
167                   Spain   
60                   France   
156                  Serbia   
96                   Latvia   
18                  Belgium   
1                   Albania   
101               Lithuania   
114                Mongolia   
140                  Poland   
123             Netherlands   
23   Bosnia and Herzegovina   
133                Pakistan   
11                  Austria   
14                  Bahrain   

                                              vaccines  no_of_vaccines  
185  [Oxford/AstraZeneca, Pfizer/BioNTech, Sinophar...               5  
111  [CanSino, Oxford/AstraZeneca, Pfizer/BioNTech,...               5  
76   [Moderna, Oxford/AstraZeneca, Pfizer/BioNTech,...               5  
97   [Oxford/AstraZeneca, Pfizer/BioNTech, Sinophar...               4  
85   [Johnson&Johnson, Moderna, Oxford/AstraZeneca,...               4  
42   [Johnson&Johnson, Moderna, Oxford/AstraZeneca,...               4  
167  [Johnson&Johnson, Moderna, Oxford/AstraZeneca,...               4  
60   [Johnson&Johnson, Moderna, Oxford/AstraZeneca,...               4  
156  [Oxford/AstraZeneca, Pfizer/BioNTech, Sinophar...               4  
96   [Johnson&Johnson, Moderna, Oxford/AstraZeneca,...               4  
18   [Johnson&Johnson, Moderna, Oxford/AstraZeneca,...               4  
1    [Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac,...               4  
101  [Johnson&Johnson, Moderna, Oxford/AstraZeneca,...               4  
114  [Oxford/AstraZeneca, Pfizer/BioNTech, Sinophar...               4  
140  [Johnson&Johnson, Moderna, Oxford/AstraZeneca,...               4  
123  [Johnson&Johnson, Moderna, Oxford/AstraZeneca,...               4  
23   [Oxford/AstraZeneca, Pfizer/BioNTech, Sinovac,...               4  
133   [CanSino, Sinopharm/Beijing, Sinovac, Sputnik V]               4  
11   [Johnson&Johnson, Moderna, Oxford/AstraZeneca,...               4  
14   [Oxford/AstraZeneca, Pfizer/BioNTech, Sinophar...               4

Number of vaccines used in Egypt: 

In [95]:
country_vaccine[country_vaccine['country']=='Egypt']

country                                 vaccines  no_of_vaccines
49   Egypt  [Oxford/AstraZeneca, Sinopharm/Beijing]               2

Another represenatation of the vaccines used in each country: 

In [96]:
df = pd.DataFrame(columns=vaccineList)
df['country']= train['country'].unique()
df.reset_index(drop=True, inplace=True)
df

Oxford/AstraZeneca Pfizer/BioNTech Sinovac Sputnik V Sinopharm/Beijing  \
0                  NaN             NaN     NaN       NaN               NaN   
1                  NaN             NaN     NaN       NaN               NaN   
2                  NaN             NaN     NaN       NaN               NaN   
3                  NaN             NaN     NaN       NaN               NaN   
4                  NaN             NaN     NaN       NaN               NaN   
..                 ...             ...     ...       ...               ...   
190                NaN             NaN     NaN       NaN               NaN   
191                NaN             NaN     NaN       NaN               NaN   
192                NaN             NaN     NaN       NaN               NaN   
193                NaN             NaN     NaN       NaN               NaN   
194                NaN             NaN     NaN       NaN               NaN   

    Johnson&Johnson Moderna Sinopharm/Wuhan Covaxin CanSino EpiVacCorona  \
0               NaN     NaN             NaN     NaN     NaN          NaN   
1               NaN     NaN             NaN     NaN     NaN          NaN   
2               NaN     NaN             NaN     NaN     NaN          NaN   
3               NaN     NaN             NaN     NaN     NaN          NaN   
4               NaN     NaN             NaN     NaN     NaN          NaN   
..              ...     ...             ...     ...     ...          ...   
190             NaN     NaN             NaN     NaN     NaN          NaN   
191             NaN     NaN             NaN     NaN     NaN          NaN   
192             NaN     NaN             NaN     NaN     NaN          NaN   
193             NaN     NaN             NaN     NaN     NaN          NaN   
194             NaN     NaN             NaN     NaN     NaN          NaN   

         country  
0    Afghanistan  
1        Albania  
2        Algeria  
3        Andorra  
4         Angola  
..           ...  
190    Venezuela  
191      Vietnam  
192        Wales  
193       Zambia  
194     Zimbabwe  

[195 rows x 12 columns]

In [97]:
countries=df['country'].unique()

In [98]:
count=0
for i in countries:
    for j in count_vaccines[count]:
        df.at[df['country']==i,j]=1
    count+=1

In [99]:
df.set_index('country',inplace=True)
# df.reset_index()
df

Oxford/AstraZeneca Pfizer/BioNTech Sinovac Sputnik V  \
country                                                            
Afghanistan                  1             NaN     NaN       NaN   
Albania                      1               1       1         1   
Algeria                    NaN             NaN     NaN         1   
Andorra                      1               1     NaN       NaN   
Angola                       1             NaN     NaN       NaN   
...                        ...             ...     ...       ...   
Venezuela                  NaN             NaN     NaN         1   
Vietnam                      1             NaN     NaN       NaN   
Wales                        1               1     NaN       NaN   
Zambia                       1             NaN     NaN       NaN   
Zimbabwe                   NaN             NaN     NaN       NaN   

            Sinopharm/Beijing Johnson&Johnson Moderna Sinopharm/Wuhan Covaxin  \
country                                                                         
Afghanistan               NaN             NaN     NaN             NaN     NaN   
Albania                   NaN             NaN     NaN             NaN     NaN   
Algeria                   NaN             NaN     NaN             NaN     NaN   
Andorra                   NaN             NaN     NaN             NaN     NaN   
Angola                    NaN             NaN     NaN             NaN     NaN   
...                       ...             ...     ...             ...     ...   
Venezuela                 NaN             NaN     NaN             NaN     NaN   
Vietnam                   NaN             NaN     NaN             NaN     NaN   
Wales                     NaN             NaN       1             NaN     NaN   
Zambia                    NaN             NaN     NaN             NaN     NaN   
Zimbabwe                    1             NaN     NaN             NaN     NaN   

            CanSino EpiVacCorona  
country                           
Afghanistan     NaN          NaN  
Albania         NaN          NaN  
Algeria         NaN          NaN  
Andorra         NaN          NaN  
Angola          NaN          NaN  
...             ...          ...  
Venezuela       NaN          NaN  
Vietnam         NaN          NaN  
Wales           NaN          NaN  
Zambia          NaN          NaN  
Zimbabwe        NaN          NaN  

[195 rows x 11 columns]

In [100]:
x=df.count().sort_values(ascending=False)

Identifying the number of country that use each vaccine:

In [101]:
fig,ax = plt.subplots(figsize=(20,10))
ax = sns.barplot(x = x.index, y =x.values, color='lightgreen')
ax.set_title("How many countries use a certain vaccine", fontsize=20)
ax.set_xlabel("Vaccine", fontsize=15)
ax.set_ylabel("n", fontsize=15)

plt.show()

We can deduce that Oxford/AstraZeneca is the most commonly used. 

# **In which country is the vaccination more advanced?**

In [102]:
#Top countries with most vaccinations
train.groupby("country")["total_vaccinations"].max().sort_values(ascending= False).head(20)

country
China                   275338000.0
United States           245591469.0
India                   154211511.0
United Kingdom           49834997.0
Brazil                   43173356.0
England                  41730517.0
Germany                  30147919.0
Turkey                   23006734.0
France                   22284791.0
Italy                    20755863.0
Indonesia                20172516.0
Russia                   20070743.0
Mexico                   18326726.0
Spain                    16364595.0
Chile                    14848435.0
Canada                   13825476.0
Poland                   11877060.0
United Arab Emirates     10634693.0
Israel                   10458345.0
Saudi Arabia              9557948.0
Name: total_vaccinations, dtype: float64

In [103]:
#barplot visualization of top countries with most vaccinations
x= train.groupby("country")["total_vaccinations"].mean().sort_values(ascending= False).head(20)
sns.set_style("whitegrid")
plt.figure(figsize= (8,8))
ax= sns.barplot(x=x.values,y=x.index)
ax.set_xlabel("Total vaccinations(mean)")
plt.show()

In [104]:
#Countries with best daily average vaccinations
x= train.groupby("country")["daily_vaccinations"].mean().sort_values(ascending= False).head(20)
#daily vaccinations barplot
plt.figure(figsize= (8,8))
ax= sns.barplot(x=x.values,y=x.index)

ax.set_xlabel("daily vaccinations(avg)")
plt.show()

We can deduce that China, United States, India, United Kingdom, and Brazil are doing great efforts in vaccination.

Timeline showing the change in daily vaccination for each country: 

In [105]:
# t=train.country.isin(t['country'])
fig = px.line(train, x = 'date', y ='daily_vaccinations', color = 'country')
fig.update_layout(
    title={
            'text' : "Daily vaccination trend",
            'y':0.95,
            'x':0.5
        },
    xaxis_title="Date",
    yaxis_title="Daily Vaccinations"
)

fig.show()

Mean number of people vaccinated and fully vaccinated in the top 5 countries: 

In [106]:
people_vaccinated_country = train.groupby("country").max().sort_values(by="people_vaccinated",ascending=False)[['people_vaccinated','people_fully_vaccinated']].head(5)
people_vaccinated_country.head()

people_vaccinated  people_fully_vaccinated
country                                                   
United States         147047012.0              104774652.0
India                 126704151.0               27507360.0
United Kingdom         34505380.0               15329617.0
Brazil                 29421191.0               13752165.0
England                28895159.0               12835358.0

Percentage of people who are fully vaccinated: 

In [107]:
people_vaccinated_country.reset_index()
people_vaccinated_country['people_fully_vaccinated/%']= (people_vaccinated_contry['people_fully_vaccinated']/people_vaccinated_contry['people_vaccinated'])*100
people_vaccinated_country.sort_values(by="people_fully_vaccinated/%", ascending=False)

people_vaccinated  people_fully_vaccinated  \
country                                                      
United States         147047012.0              104774652.0   
Brazil                 29421191.0               13752165.0   
United Kingdom         34505380.0               15329617.0   
England                28895159.0               12835358.0   
India                 126704151.0               27507360.0   

                people_fully_vaccinated/%  
country                                    
United States                   71.252486  
Brazil                          46.742380  
United Kingdom                  44.426744  
England                         44.420444  
India                           21.709912

To measure the vaccination progress in a country, we will perform analysis on **Total vaccinations per hundred**, **Total number of people vaccinated per hundred**, 
**Total number of people fully vaccinated per hundred**, and **Daily vaccinations per million**. 

Total vaccinations per hundred - ratio (in percent) between vaccination number and total population up to the date in the country;
Total number of people vaccinated per hundred - ratio (in percent) between population immunized and total population up to the date in the country;
Total number of people fully vaccinated per hundred - ratio (in percent) between population fully immunized and total population up to the date in the country;

In [108]:
#First five countrys
vaccinated_per_hundred = train.groupby("country").max().sort_values(by="total_vaccinations_per_hundred",ascending=False)
vaccinated_per_hundred.head()[['total_vaccinations_per_hundred','people_vaccinated_per_hundred','people_fully_vaccinated_per_hundred']]

total_vaccinations_per_hundred  \
country                                                
Gibraltar                                     211.08   
Seychelles                                    128.98   
Falkland Islands                              126.53   
Israel                                        120.83   
United Arab Emirates                          107.53   

                      people_vaccinated_per_hundred  \
country                                               
Gibraltar                                    111.32   
Seychelles                                    68.82   
Falkland Islands                              75.57   
Israel                                        62.43   
United Arab Emirates                          51.38   

                      people_fully_vaccinated_per_hundred  
country                                                    
Gibraltar                                           99.76  
Seychelles                                          60.16  
Falkland Islands                                    50.96  
Israel                                              58.40  
United Arab Emirates                                38.79

In [109]:
labels = vaccinated_per_hundred.head(20).index
total_vaccinations_per_hundred = vaccinated_per_hundred['total_vaccinations_per_hundred'].head(20)
people_vaccinated_per_hundred = vaccinated_per_hundred['people_vaccinated_per_hundred'].head(20)
people_fully_vaccinated_per_hundred = vaccinated_per_hundred['people_fully_vaccinated_per_hundred'].head(20)

x = np.arange(len(labels)) 

width = 0.40   
fig, ax = plt.subplots(figsize=(20,12))

ax.bar(labels, total_vaccinations_per_hundred , width, label='Total vaccinations per hundred')
ax.bar(labels, people_vaccinated_per_hundred, width,label='People vaccinated per hundred')
ax.bar(labels, people_fully_vaccinated_per_hundred, width,label='People fully vaccinated per hundred')

ax.set_ylabel('n / hundread',fontsize=18)
ax.set_xlabel('Country',fontsize=18)
ax.set_title("Vaccinations per hundred per country - 20th",fontsize=22)
ax.set_xticks(x)
ax.set_xticklabels(labels,fontsize=14,rotation = 70)

ax.legend()

plt.show()

Measuring the gap between the number of people vaccinated and the people fully vaccinated.

In [110]:
#First five countrys

vaccinated_per_hundred = train.groupby("country").max().sort_values(by="total_vaccinations_per_hundred",ascending=False)
vaccinated_per_hundred=vaccinated_per_hundred[['total_vaccinations_per_hundred','people_vaccinated_per_hundred','people_fully_vaccinated_per_hundred']].head()
vaccinated_per_hundred['people_fully_vaccinated_from_total_vaccinated/%']= (vaccinated_per_hundred['people_fully_vaccinated_per_hundred']/vaccinated_per_hundred['people_vaccinated_per_hundred'])*100

vaccinated_per_hundred

total_vaccinations_per_hundred  \
country                                                
Gibraltar                                     211.08   
Seychelles                                    128.98   
Falkland Islands                              126.53   
Israel                                        120.83   
United Arab Emirates                          107.53   

                      people_vaccinated_per_hundred  \
country                                               
Gibraltar                                    111.32   
Seychelles                                    68.82   
Falkland Islands                              75.57   
Israel                                        62.43   
United Arab Emirates                          51.38   

                      people_fully_vaccinated_per_hundred  \
country                                                     
Gibraltar                                           99.76   
Seychelles                                          60.16   
Falkland Islands                                    50.96   
Israel                                              58.40   
United Arab Emirates                                38.79   

                      people_fully_vaccinated_from_total_vaccinated/%  
country                                                                
Gibraltar                                                   89.615523  
Seychelles                                                  87.416449  
Falkland Islands                                            67.434167  
Israel                                                      93.544770  
United Arab Emirates                                        75.496302